<a href="https://colab.research.google.com/github/frank-heijnen/QIG-RM/blob/Ivan/index_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === Core imports ===
import io
import requests
import numpy as np
import pandas as pd
from datetime import date, timedelta
import statsmodels.api as sm
from scipy.special import expit
from sklearn.metrics import roc_auc_score
FRED_SERIES_CSV = "https://fred.stlouisfed.org/graph/fredgraph.csv?id={sid}"

# ---------- 0) Core FRED fetcher ----------
def fetch_fred_series(series_id: str) -> pd.Series:
    """Return a FRED series as a clean pandas Series (tz-naive index)."""
    url = FRED_SERIES_CSV.format(sid=series_id)
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    df = pd.read_csv(io.StringIO(r.text))
    df.columns = [c.strip().lstrip("\ufeff") for c in df.columns]

    # detect date/value columns
    date_col = next((c for c in ["DATE", "date", "observation_date"] if c in df.columns), df.columns[0])
    val_col = next((c for c in df.columns if c.upper().startswith(series_id.upper())), df.columns[-1])

    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col]).set_index(date_col).sort_index()
    s = pd.to_numeric(df[val_col], errors="coerce").dropna()
    s.name = series_id
    s.index = pd.to_datetime(s.index).tz_localize(None)
    return s

# ---------- 1) CPI & inflation history ----------
CPI_SERIES_ID = "CPIAUCSL"

def fetch_cpi() -> pd.Series:
    return fetch_fred_series(CPI_SERIES_ID)

def summarize_inflation(cpi: pd.Series) -> pd.DataFrame:
    last_date = cpi.index[-1]
    last_val = cpi.iloc[-1]
    yoy = (cpi / cpi.shift(12) - 1.0) * 100.0
    last_yoy = yoy.dropna().iloc[-1]
    mom_ann = (np.log(cpi) - np.log(cpi.shift(1))) * 1200.0
    last_mom_ann = mom_ann.dropna().iloc[-1]
    return pd.DataFrame({
        "Last CPI": [last_val],
        "YoY %": [last_yoy],
        "MoM annualized %": [last_mom_ann],
        "As of": [last_date.date()],
    })

def fetch_inflation_history(start: str = "2009-01-01", end: str | None = None, dropna_rows: bool = False) -> pd.DataFrame:
    if end is None:
        end = date.today().isoformat()
    cpi_full = fetch_cpi().rename("CPI")
    yoy = (cpi_full / cpi_full.shift(12) - 1.0) * 100.0
    mom = (cpi_full / cpi_full.shift(1) - 1.0) * 100.0
    mom_ann = (np.log(cpi_full) - np.log(cpi_full.shift(1))) * 1200.0
    df = pd.concat([cpi_full, yoy.rename("YoY %"), mom.rename("MoM %"), mom_ann.rename("MoM annualized %")], axis=1).loc[start:end]
    if dropna_rows:
        df = df.dropna(subset=["CPI"])
    df.index.name = "observation_date"
    return df

# ---------- 2) PSAVERT + CDSP ----------
def fetch_psavert_cdsp(start: str = "2009-01-01", end: str | None = None) -> pd.DataFrame:
    if end is None:
        end = date.today().isoformat()
    psavert = fetch_fred_series("PSAVERT").rename("PSAVERT").loc[start:end]
    cdsp = fetch_fred_series("CDSP").rename("CDSP").loc[start:end]
    df = pd.concat([psavert, cdsp], axis=1).sort_index()
    df.index.name = "observation_date"
    return df

# ---------- 3) FX monthly (USD per foreign unit) ----------
def fetch_fx_monthly(start: str = "2009-01-01", end: str | None = None, agg: str = "mean", stamp: str = "MS") -> pd.DataFrame:
    if end is None:
        end = date.today().isoformat()
    chf_per_usd = fetch_fred_series("DEXSZUS")  # CHF / USD
    usd_per_eur = fetch_fred_series("DEXUSEU")  # USD / EUR
    dkk_per_usd = fetch_fred_series("EXDNUS")   # DKK / USD

    usd_per_chf = (1.0 / chf_per_usd).replace([np.inf, -np.inf], np.nan).rename("USDCHF")
    usd_per_eur = usd_per_eur.rename("USDEUR")
    usd_per_dkk = (1.0 / dkk_per_usd).replace([np.inf, -np.inf], np.nan).rename("USDDKK")

    agg_fn = {"mean": "mean", "last": "last", "first": "first"}[agg]
    chf_m = usd_per_chf.loc[start:end].resample("MS").agg(agg_fn)
    eur_m = usd_per_eur.loc[start:end].resample("MS").agg(agg_fn)
    dkk_m = usd_per_dkk.loc[start:end].resample("MS").agg(agg_fn)
    df = pd.concat([chf_m, eur_m, dkk_m], axis=1).dropna(how="all")

    per = df.index.to_period("M")
    df.index = per.to_timestamp(how=("start" if stamp == "MS" else "end"))
    if stamp == "ME":
        df.index = df.index.normalize()
    df.index.name = "observation_date"
    return df

# ---------- 4) Treasury yields (monthly averages) ----------
def fetch_bond_yields(start: str = "2009-01-01", end: str | None = None,
                      series: dict[str, str] | None = None, stamp: str = "MS") -> pd.DataFrame:
    if end is None:
        end = date.today().isoformat()
    if series is None:
        series = {"DGS7": "DGS7", "DGS10": "DGS10", "DGS30": "DGS30"}
    cols = []
    for name, sid in series.items():
        s = fetch_fred_series(sid).rename(name)
        s = s.loc[start:end].resample("MS").mean()
        cols.append(s)
    df = pd.concat(cols, axis=1).dropna(how="all").sort_index()
    per = df.index.to_period("M")
    df.index = per.to_timestamp(how=("start" if stamp == "MS" else "end"))
    if stamp == "ME":
        df.index = df.index.normalize()
    df.index.name = "observation_date"
    return df

# ---------- 5) Generic monthly combiner (future-proof) ----------
def to_monthly(obj, how: str = "mean", stamp: str = "MS"):
    if stamp not in ("MS", "ME"):
        raise ValueError("stamp must be 'MS' or 'ME'")
    agg_fn = {"mean": "mean", "last": "last", "first": "first"}[how]
    if isinstance(obj, pd.Series):
        x = obj.copy()
        x.index = pd.to_datetime(x.index)
        x = x.resample("MS").agg(agg_fn)
        if stamp == "ME":
            x.index = x.index.to_period("M").to_timestamp(how="end").normalize()
        return x
    elif isinstance(obj, pd.DataFrame):
        X = obj.copy()
        X.index = pd.to_datetime(X.index)
        X = X.resample("MS").agg(agg_fn)
        if stamp == "ME":
            X.index = X.index.to_period("M").to_timestamp(how="end").normalize()
        return X
    else:
        raise TypeError("obj must be a pandas Series or DataFrame")

def combine_monthly(items, *, start="2009-01-01", end=None, stamp="MS",
                    how="mean", drop_partial_last=True, round_decimals=None, join="outer"):
    if end is None:
        end = date.today().isoformat()
    frames = []
    if isinstance(items, dict):
        for name, obj in items.items():
            m = to_monthly(obj, how=how, stamp=stamp)
            if isinstance(m, pd.Series):
                m = m.to_frame()
            m.columns = [f"{name}_{c}" if c not in ("", None) else name for c in m.columns]
            frames.append(m)
    else:
        for obj in items:
            m = to_monthly(obj, how=how, stamp=stamp)
            if isinstance(m, pd.Series):
                m = m.to_frame()
            frames.append(m)
    df = pd.concat(frames, axis=1, join=join).sort_index().loc[start:end]
    df.index.name = "observation_date"
    if drop_partial_last and not df.empty:
        last_idx = df.index[-1]
        if (last_idx.year, last_idx.month) == (date.today().year, date.today().month):
            df = df.iloc[:-1]
    if round_decimals is not None:
        df = df.round(round_decimals)
    return df


In [ ]:
infl_hist      = fetch_inflation_history(start="2009-01-01")           # CPI + YoY/MoM
ps_cd          = fetch_psavert_cdsp(start="2009-01-01")                # PSAVERT + CDSP
yields_mavg    = fetch_bond_yields(start="2009-01-01")                 # DGS7/10/30 monthly avg
fx_monthly     = fetch_fx_monthly(start="2009-01-01", agg="mean")      # USDCHF/USDEUR/USDDKK

df_big = combine_monthly(
    {"infl": infl_hist, "pc": ps_cd, "yld": yields_mavg, "fx": fx_monthly},
    start="2009-01-01",
    stamp="MS",
    how="mean",
    round_decimals=4,
    join="outer",
)


In [ ]:
df_big

,infl_CPI,infl_YoY %,infl_MoM %,infl_MoM annualized %,pc_PSAVERT,pc_CDSP,yld_DGS7,yld_DGS10,yld_DGS30,fx_USDCHF,fx_USDEUR,fx_USDDKK
observation_date,,,,,,,,,,,,
2009-01-01,211.933,-0.1136,0.2531,3.0331,5.9,6.8614,1.9780,2.5175,3.1280,0.8880,1.3244,0.1777
2009-02-01,212.705,0.0085,0.3643,4.3633,5.4,NaN,2.3032,2.8700,3.5868,0.8592,1.2797,0.1717
2009-03-01,212.495,-0.4465,-0.0987,-1.1853,5.8,NaN,2.4177,2.8195,3.6432,0.8658,1.3050,0.1750
2009-04-01,212.709,-0.5763,0.1007,1.2079,6.5,6.7050,2.4652,2.9271,3.7600,0.8711,1.3199,0.1772
2009-05-01,213.022,-1.0158,0.1471,1.7645,7.8,NaN,2.8105,3.2930,4.2270,0.9031,1.3646,0.1832
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-01,320.321,2.3337,0.2209,2.6478,5.7,5.3574,4.0848,4.2790,4.7110,1.2002,1.1232,0.1504
2025-05-01,320.580,2.3759,0.0809,0.9699,5.2,NaN,4.2181,4.4238,4.9048,1.2050,1.1274,0.1511
2025-06-01,321.500,2.6727,0.2870,3.4388,5.0,NaN,4.1570,4.3835,4.8910,1.2298,1.1534,0.1546


In [ ]:
# US recession dummy, trough method (monthly 0/1)
usrec = fetch_fred_series("USREC").rename("USREC")  # from your generic FRED fetcher
# If you want a clean month-start index:
usrec = usrec.resample("MS").last()

In [ ]:
def _drop_constant_and_na(X: pd.DataFrame, min_std=1e-12):
    keep = []
    for c in X.columns:
        s = X[c]
        if not np.isfinite(s).any():
            continue
        if s.std(ddof=0) > min_std:
            keep.append(c)
    return X[keep]

def _drop_high_corr(X: pd.DataFrame, thresh: float = 0.98):
    """
    Drop one feature from each pair with |corr| >= thresh.
    Keeps the first occurrence; greedy but effective.
    """
    corr = X.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    drop_cols = [col for col in upper.columns if any(upper[col] >= thresh)]
    return X.drop(columns=drop_cols), drop_cols

def fit_logit_regularized(df, feature_cols, target_col="USREC",
                          corr_thresh=0.98, alpha=1.0, l1_wt=0.0):
    """
    Regularized logistic regression (ridge by default) that is robust to
    multicollinearity and separation.

    alpha: overall penalty strength
    l1_wt: 0.0 -> L2 (ridge), 1.0 -> L1 (lasso), in-between -> elastic-net
    """
    # 1) Start with predictors
    X = df[feature_cols].copy()
    y = df[target_col].astype(int).copy()

    # Clean numeric data
    X = X.replace([np.inf, -np.inf], np.nan)
    X = X.dropna(axis=1, how="all")
    # Drop any rows with missing target or all-missing features
    m = y.notna()
    X, y = X.loc[m], y.loc[m]
    # Simple row-wise fill for any remaining NaNs (from quarterly series etc.)
    X = X.fillna(method="ffill").fillna(method="bfill")

    # 2) Remove constant/near-constant columns
    X = _drop_constant_and_na(X)

    # 3) Remove highly correlated columns
    X, dropped_corr = _drop_high_corr(X, thresh=corr_thresh)

    kept_features = list(X.columns)

    # 4) Standardize (z-score) for stability & interpretable betas
    mu = X.mean()
    sd = X.std(ddof=0).replace(0, 1.0)  # guard divide-by-zero
    Xz = (X - mu) / sd

    # 5) Add intercept
    Xz = sm.add_constant(Xz, has_constant='add')

    # 6) Regularized fit (prevents singular matrix / separation)
    model = sm.Logit(y, Xz)
    res = model.fit_regularized(alpha=alpha, L1_wt=l1_wt, maxiter=2000, disp=False)

    # 7) Scores & probabilities
    linpred = np.dot(Xz, res.params.values)   # β0 + β·X

    prob = expit(linpred)   # numerically stable sigmoid

    scores = pd.DataFrame({
        "RecessionIndex": linpred,       # linear score (can be your index)
        "RecessionProb": prob,           # 0..1 probability
    }, index=Xz.index)

    # 8) Quick quality metric
    auc = roc_auc_score(y.loc[scores.index], scores["RecessionProb"])

    out = {
        "result": res,
        "kept_features": kept_features,
        "dropped_high_corr": dropped_corr,
        "feature_means": mu,
        "feature_stds": sd,
        "auc": auc,
        "Xz": Xz,
        "y": y.loc[Xz.index],
        "scores": scores
    }
    return out


In [ ]:
# Build modeling frame as you did (lag 1, ffill), then:
out = fit_logit_regularized(df_model, features, target_col="USREC",
                            corr_thresh=0.98, alpha=1.0, l1_wt=0.0)  # ridge

print("AUC:", out["auc"])
print("Kept features:", out["kept_features"])
print("Dropped for high corr:", out["dropped_high_corr"])
print(out["result"].summary())   # coeffs (on standardized features)

scores = out["scores"]
# e.g., threshold 0.5 classification:
y_hat = (scores["RecessionProb"] >= 0.5).astype(int)


AUC: 0.9955357142857143
Kept features: ['infl_CPI', 'infl_YoY %', 'infl_MoM %', 'pc_PSAVERT', 'pc_CDSP', 'yld_DGS7', 'yld_DGS10', 'yld_DGS30', 'fx_USDCHF', 'fx_USDEUR']
Dropped for high corr: ['infl_MoM annualized %', 'fx_USDDKK']
                           Logit Regression Results                           
Dep. Variable:                  USREC   No. Observations:                  199
Model:                          Logit   Df Residuals:                      193
Method:                           MLE   Df Model:                            5
Date:                Sun, 28 Sep 2025   Pseudo R-squ.:                  0.6858
Time:                        18:26:34   Log-Likelihood:                -9.5213
converged:                       True   LL-Null:                       -30.307
Covariance Type:            nonrobust   LLR p-value:                 7.191e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------

/tmp/ipython-input-3640877333.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = X.fillna(method="ffill").fillna(method="bfill")
